# Побудова нейронної мережі для розпізнавання шрифтів

### Підготовка наборів

Перед початком побудови нейронної мережі, слід підготувати тренувальний та валідаційні набори. Для цього було обрано 5 різних наборів шрифтів для кожної сімейства шрифтів (serif, sans-serif, monospace, script, display, handwritten), щоб модель змогла навчитися розрізняти ключові риси між сімействами, та навчитися розрізняти їх між собою всередині сімейства. В будь-якому випадку ми будемо будувати модель розпізнавання конкретних шрифтів, а не їх сімейств.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

Для цього було створено функцію, яка завантажує набір, прибирає стовпці, які не будуть використовуватись під час навчання, створює зразок з 950 випадкових рядків набору (така кількість обумовлена тим, що деякі набори шрифтів містять таку мінімальну кількість записів і щоб в моделі не було перенавчання на окремих шрифтах, які можуть мати до 60000 рядків в наборі, ми беремо однакову кількість рядків для кожного набору шрифтів), та розподіляє їх на тренувальні та валідаційні зразки у відношенні 80:20 після чого додаємо до загальних тренувального та валідаційного наборів.

In [6]:
def create_train_test_datasets(font_paths):
    train_dataset = pd.DataFrame()
    test_dataset = pd.DataFrame()

    columns_to_drop = [
        'fontVariant', 
        'm_label', 
        'strength', 
        'italic', 
        'orientation', 
        'm_top',
        'm_left',
        'originalH',
        'originalW',
        'h',
        'w'
    ]

    for font_path in font_paths:
        dataset = pd.read_csv(font_path)
        dataset = dataset.drop(columns=columns_to_drop, errors='ignore')

        sample = dataset.sample(n=950, random_state=42)

        train_sample, test_sample = train_test_split(sample, test_size=0.2, random_state=42)

        train_dataset = pd.concat([train_dataset, train_sample], ignore_index=True)
        test_dataset = pd.concat([test_dataset, test_sample], ignore_index=True)

    return train_dataset, test_dataset

Навчання буде проводитись на таких шрифтах:

- serif:

    - BASKERVILLE
    - BELL
    - BERNARD
    - CALISTO
    - CENTAUR

- sans-serif:

    - AGENCY
    - BAUHAUS
    - BERLIN
    - BRITANNIC
    - BUXTON

- monospace

    - CONSOLAS
    - COURIER
    - MONOSPAC821
    - MONOTXT
    - SIMPLEX

- script:

    - BLACKADDER
    - BRUSH
    - COMMERCIALSCRIPT
    - EDWARDIAN
    - ENGLISH

- display:

    - BROADWAY
    - CASTELLAR
    - CURLZ
    - GUNPLAY
    - JOKERMAN

- handwritten:

    - BRADLEY
    - CHILLER
    - KRISTEN
    - RAGE
    - VINER

In [7]:
font_paths = [
    './fonts/serif/BASKERVILLE.csv',
    './fonts/serif/BELL.csv',
    './fonts/serif/BERNARD.csv',
    './fonts/serif/CALISTO.csv',
    './fonts/serif/CENTAUR.csv',

    './fonts/sans-serif/AGENCY.csv',
    './fonts/sans-serif/BAUHAUS.csv',
    './fonts/sans-serif/BERLIN.csv',
    './fonts/sans-serif/BRITANNIC.csv',
    './fonts/sans-serif/BUXTON.csv',

    './fonts/monospace/CONSOLAS.csv',
    './fonts/monospace/COURIER.csv',
    './fonts/monospace/MONOSPAC821.csv',
    './fonts/monospace/MONOTXT.csv',
    './fonts/monospace/SIMPLEX.csv',

    './fonts/script/BLACKADDER.csv',
    './fonts/script/BRUSH.csv',
    './fonts/script/COMMERCIALSCRIPT.csv',
    './fonts/script/EDWARDIAN.csv',
    './fonts/script/ENGLISH.csv',

    './fonts/display/BROADWAY.csv',
    './fonts/display/CASTELLAR.csv',
    './fonts/display/CURLZ.csv',
    './fonts/display/GUNPLAY.csv',
    './fonts/display/JOKERMAN.csv',

    './fonts/handwritten/BRADLEY.csv',
    './fonts/handwritten/CHILLER.csv',
    './fonts/handwritten/KRISTEN.csv',
    './fonts/handwritten/RAGE.csv',
    './fonts/handwritten/VINER.csv'
]

In [8]:
train_dataset, test_dataset = create_train_test_datasets(font_paths)

In [9]:
train_dataset

,font,r0c0,r0c1,r0c2,r0c3,r0c4,r0c5,r0c6,r0c7,r0c8,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,BASKERVILLE,1,1,1,1,1,74,255,255,255,...,255,255,255,255,74,1,1,1,1,1
1,BASKERVILLE,1,1,1,1,1,1,121,121,255,...,1,1,1,1,1,1,1,1,1,1
2,BASKERVILLE,1,1,154,255,255,226,183,183,183,...,183,183,183,212,255,255,255,255,255,52
3,BASKERVILLE,1,1,1,1,1,1,1,1,1,...,232,232,232,232,232,194,1,1,1,1
4,BASKERVILLE,227,243,255,255,255,255,255,255,255,...,241,227,227,227,247,255,255,255,255,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,VINER,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22796,VINER,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22797,VINER,148,255,255,255,255,255,255,255,255,...,168,168,168,168,168,168,168,183,178,1
22798,VINER,1,1,1,1,37,168,255,232,1,...,1,40,242,174,1,1,1,1,1,1


In [10]:
test_dataset

,font,r0c0,r0c1,r0c2,r0c3,r0c4,r0c5,r0c6,r0c7,r0c8,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,BASKERVILLE,1,128,255,255,128,1,1,1,1,...,128,1,1,1,1,128,255,255,1,1
1,BASKERVILLE,1,1,1,1,1,1,1,1,1,...,15,1,1,1,1,1,1,1,1,1
2,BASKERVILLE,1,1,1,1,1,1,1,1,1,...,11,255,255,255,255,252,141,53,1,1
3,BASKERVILLE,1,1,1,1,1,1,1,1,1,...,255,255,255,255,255,255,49,1,1,1
4,BASKERVILLE,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,162,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,VINER,1,1,1,1,1,1,1,29,142,...,1,14,198,206,6,1,1,1,1,1
5696,VINER,1,1,1,134,255,255,244,1,1,...,255,255,255,255,255,255,134,1,1,1
5697,VINER,1,1,1,1,1,1,1,55,182,...,13,182,255,234,13,8,1,1,1,1
5698,VINER,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


Екпортуємо створені набори в csv формат для більш зручного використання у майбутньому.

In [12]:
train_dataset.to_csv('./final/train_dataset.csv', index=False)
test_dataset.to_csv('./final/test_dataset.csv', index=False)

### Навчання моделі

Перед навчанням моделі імпортуємо створені набори, приберемо назви шрифтів, нормалізуємо та відцентруємо тренувальні дані. 

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
def preprocess_data(train_path, test_path):
    train_dataset = pd.read_csv(train_path)
    test_dataset = pd.read_csv(test_path)

    X_train = train_dataset.drop('font', axis=1).values
    y_train = train_dataset['font'].values

    X_test = test_dataset.drop('font', axis=1).values
    y_test = test_dataset['font'].values

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    mean_train = np.mean(X_train, axis=0)
    X_train_centered = X_train - mean_train
    X_test_centered = X_test - mean_train

    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    one_hot_encoder = OneHotEncoder(sparse=False)
    y_train_one_hot = one_hot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
    y_test_one_hot = one_hot_encoder.transform(y_test_encoded.reshape(-1, 1))

    return X_train_centered, X_test_centered, y_train_one_hot, y_test_one_hot